In [6]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
from math import log, sqrt
import pandas as pd
import numpy as np
import re
from sklearn.datasets import load_files 
#nltk.download('stopwords')  
import pickle  
from nltk.corpus import stopwords
%matplotlib inline  

In [7]:
mails_dataset = pd.read_csv('Dataset/trial_spam.csv', encoding = 'latin-1')
mails_dataset.head()           #show first 5 rows

,v1,v2,Unnamed: 2,Unnamed: 3
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN
1,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN
2,spam,WINNER!! As a valued network customer you have...,NaN,NaN
3,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN
4,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN


In [8]:
#drop undesirable columns
mails_dataset.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)
mails_dataset.head()

KeyError: "['Unnamed: 4'] not found in axis"

In [9]:
#Rename the columns, to make it easy to read and manipulate
mails_dataset.rename(columns = {'v1': 'label', 'v2': 'message'}, inplace = True)
mails_dataset.head()

,label,message,Unnamed: 2,Unnamed: 3
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN
1,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN
2,spam,WINNER!! As a valued network customer you have...,NaN,NaN
3,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN
4,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN


In [10]:
mails_dataset['label'].value_counts()  #count number of each label

spam    100
ham     100
Name: label, dtype: int64

In [11]:
total_mails = mails_dataset.shape[0]            #total number on instances in our dataset
total_mails

200

In [12]:
#convert the data into lower case
mails_dataset['message'] =  mails_dataset['message'].str.lower()
mails_dataset.head()

,label,message,Unnamed: 2,Unnamed: 3
0,spam,free entry in 2 a wkly comp to win fa cup fina...,NaN,NaN
1,spam,freemsg hey there darling it's been 3 week's n...,NaN,NaN
2,spam,winner!! as a valued network customer you have...,NaN,NaN
3,spam,had your mobile 11 months or more? u r entitle...,NaN,NaN
4,spam,"six chances to win cash! from 100 to 20,000 po...",NaN,NaN


In [13]:
mails_dataset['label'] = mails_dataset['label'].map({'ham': 0, 'spam': 1})
mails_dataset.head()

,label,message,Unnamed: 2,Unnamed: 3
0,1,free entry in 2 a wkly comp to win fa cup fina...,NaN,NaN
1,1,freemsg hey there darling it's been 3 week's n...,NaN,NaN
2,1,winner!! as a valued network customer you have...,NaN,NaN
3,1,had your mobile 11 months or more? u r entitle...,NaN,NaN
4,1,"six chances to win cash! from 100 to 20,000 po...",NaN,NaN
